# Title: [No idea]

## This project aims to brace up my knowledge with Data Science till now, involving the basics DS libraries: Numpy, Pandas e Matplotlib.

The following dataset will be explored:

* **Data Source:** MovieLens web site (filename: ml-25m.zip)
* **Location:** https://grouplens.org/datasets/movielens/

# Importing Libraries

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Exploring data

## Using UNIX to take a look on the data

In [ ]:
!ls ./movielens

In [ ]:
#!head -5 ./movielens/movies.csv
#!head -5 ./movielens/genome-scores.csv
#!head -5 ./movielens/genome-tags.csv
#!head -5 ./movielens/links.csv
#!head -5 ./movielens/ratings.csv
!head -5 ./movielens/tags.csv

# Using Pandas to explore the data

Reading CSVs

In [110]:
movie = pd.read_csv('./movielens/movies.csv', sep=',')
rating = pd.read_csv('./movielens/ratings.csv', sep=',')
tag = pd.read_csv('./movielens/tags.csv', sep=',')

Timestamp into datetime

In [44]:
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

Taking a look to the genres

In [84]:
movie_genres = movie['genres'].str.split('|', expand=True)

In [114]:
movie_genres_array = np.array(movie_genres)
movie_genres_single = pd.Series(movie_genres_array.reshape(movie_genres_array.size))

In [111]:
movie_genres_single.value_counts().sort_values(ascending=False)
# There's 19 unique genres

Drama                 25606
Comedy                16870
Thriller               8654
Romance                7719
Action                 7348
Horror                 5989
Documentary            5605
Crime                  5319
(no genres listed)     5062
Adventure              4145
Sci-Fi                 3595
Children               2935
Animation              2929
Mystery                2925
Fantasy                2731
War                    1874
Western                1399
Musical                1054
Film-Noir               353
IMAX                    195
dtype: int64

In [113]:
unique_genres = movie_genres_single.sort_values(ascending=True).unique()[1:-1]
unique_genres
# Excluding '(no genres listed)' and 'None'

array(['Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

Joining movies with ratings datasets

In [45]:
movie_rating = movie.merge(rating[['rating','timestamp','movieId']], on='movieId', how='left')
#movie_tag = movie.merge(tag[['tag','timestamp','movieId']], on='movieId', how='left') 

# Possible questions

* Improvement of a genre by the eyes of the users as time passed (avg rating each year/month) - (total = 19 genres)
* behavior of a specific movie since its release 


# Data Cleaning